In [2]:
import scipy.io

In [3]:
data = scipy.io.loadmat('cars_annos.mat')
class_names = data['class_names']
f_class = open('./label_map1.txt','w')
num = 1
for j in range(class_names.shape[1]):
    class_name = str(class_names[0,j][0]).replace(' ','_')
    #print(num,class_name)
    f_class.write(str(num) + ' ' + class_name + '\n')
    num = num + 1
f_class.close()

In [4]:
annotations = data['annotations']
f_train = open('./train.txt','w') 
for i in range(annotations.shape[1]):    
    num = int(annotations[0,i][5])    
    num = str(num)       
    f_train.write(num+'\n') 
f_train.close()

In [5]:
import pandas as pd
label = pd.read_csv('train.txt',header = None)
label.loc[:,0].value_counts()

119    136
79      97
161     96
167     95
56      93
      ... 
175     61
64      59
158     58
99      55
136     48
Name: 0, Length: 196, dtype: int64

In [6]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import Dataset

from torch.optim import lr_scheduler
from torch.autograd import Variable

import os 
import time

from PIL import Image

In [7]:
def default_loader(path):
    try:
        img = Image.open(path)
        return img.convert('RGB')
    except:
        print('Cannot read image:{}'.fromat(path))

In [8]:
txt_path = './train.txt'
with open(txt_path) as input_file:
            lines = input_file.readlines()
            #print(lines)
            img_labels = list(map(int,[line.strip() for line in lines]))
            img_label = img_labels[:8144]
            print(len(img_label))

8144


In [9]:
#定义数据集类，读取数据和标签
class Mydataset(Dataset):
    def __init__(self,img_path,txt_path,dataset='',data_transforms = None,loader = default_loader):
        with open(txt_path) as input_file:
            lines = input_file.readlines()
            self.img_label = list(map(int,[line.strip() for line in lines]))#读入图片标签
        self.img_name = []
        for root,dirs,files in os.walk(img_path):            
            for name in sorted(files):                
                self.img_name.append(os.path.join(img_path,name))
        self.data_transforms = data_transforms 
        self.dataset = dataset
        self.loader = loader
        
    def __len__(self):
        return len(self.img_name)
    
    def __getitem__(self,item):
        img_name = self.img_name[item]
        label = self.img_label[item]
        img = self.loader(img_name)
        
        if self.data_transforms is not None:
            try:
                img = self.data_transforms[self.dataset](img)
            except:
                print('Cannot transform images:{}'.format(img_name))
        return img,label

In [42]:
def train_model(model,criterion,optimizer,scheduler,num_epochs,use_gpu):
    since = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        begin_time = time.time()
        print('Epoch {}/{}'.format(epoch,num_epochs-1))
        print('-'*10)
        
        for phase in ['train']:
            count_batch = 0
            if phase =='train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)
            running_loss = 0.0
            running_corrects = 0.0
            
            #遍历数据
            for data in dataloders[phase]:
                count_batch +=1
                inputs,labels = data
                
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                    
                #梯度归0   
                optimizer.zero_grad()
                
                #前向传播
                outputs = model(inputs)
                _,preds = torch.max(outputs.data,1)
                loss = criterion(outputs,labels)
                
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)
                
                #打印每50次迭代的结果
                if count_batch%50 == 0:
                    batch_loss = running_loss / (batch_size*count_batch)
                    batch_acc = running_corrects / (batch_size*count_batch)
                    print('{} Epoch [{}] Batch [{}] Loss: {:.4f} Acc: {:.4f} Time: {:.4f}s'. \
                          format(phase, epoch, count_batch, batch_loss, batch_acc, time.time()-begin_time))
                    begin_time = time.time()
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            #保存模型
            if phase == 'train':
                if not os.path.exists('output'):
                    os.makedirs('output')
                torch.save(model, 'output/resnet_epoch{}.pkl'.format(epoch))
                
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # print('Best val Acc: {:4f}'.format(best_acc))
    print('Best acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [43]:
#开始加载数据
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop((256,256),interpolation=3),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])}
use_gpu = torch.cuda.is_available()
print(use_gpu)

batch_size = 32
num_class = 197

image_datasets = {x: Mydataset(img_path='./'+x,
                                    txt_path='./'+ x + '.txt',
                                    data_transforms=data_transforms,
                                    dataset=x) for x in ['train']}

#将图片数据和标签放入tensor中
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=batch_size,
                                                 shuffle=True) for x in ['train']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}

True


In [44]:
dataloders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2ec062cd278>}

In [45]:
#
model_ft = models.resnet18(pretrained=False)
num_ftrs = model_ft.fc.in_features
print(num_ftrs)

512


16

In [46]:
model_ft.fc = nn.Linear(num_ftrs, num_class)
#model_ft = model_ft.cuda()
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.005, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.2)

model_ft = train_model(model=model_ft,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=10,
                           use_gpu=False)

torch.save(model_ft,"output/best_resnet.pkl")

Epoch 0/9
----------
train Epoch [0] Batch [50] Loss: 0.1689 Acc: 0.0081 Time: 184.1120s
train Epoch [0] Batch [100] Loss: 0.1684 Acc: 0.0081 Time: 156.1392s
train Epoch [0] Batch [150] Loss: 0.1684 Acc: 0.0088 Time: 152.5495s
train Epoch [0] Batch [200] Loss: 0.1683 Acc: 0.0091 Time: 146.8067s
train Epoch [0] Batch [250] Loss: 0.1684 Acc: 0.0095 Time: 146.4767s
train Loss: 0.1687 Acc: 0.0093
Epoch 1/9
----------
train Epoch [1] Batch [50] Loss: 0.1680 Acc: 0.0106 Time: 139.1851s
train Epoch [1] Batch [100] Loss: 0.1683 Acc: 0.0094 Time: 139.6777s
train Epoch [1] Batch [150] Loss: 0.1682 Acc: 0.0100 Time: 152.9628s
train Epoch [1] Batch [200] Loss: 0.1684 Acc: 0.0089 Time: 173.1314s
train Epoch [1] Batch [250] Loss: 0.1684 Acc: 0.0088 Time: 160.2607s
train Loss: 0.1687 Acc: 0.0090
Epoch 2/9
----------
train Epoch [2] Batch [50] Loss: 0.1680 Acc: 0.0094 Time: 138.8761s
train Epoch [2] Batch [100] Loss: 0.1681 Acc: 0.0094 Time: 137.9469s
train Epoch [2] Batch [150] Loss: 0.1682 Acc: 0.00